In [0]:
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

In [2]:
!pip install sddk
import sddk

In [3]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


# Read data

In [0]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_merged.json", "df", conf)

In [8]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4
0,/text/237766?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials",10,Regions\nAttica (IG I-III),"Clinton, Sacred Officials 10",None,None,46.0,5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20\n\n\n\n\...,[․․․․․12․․․․․] ὀβολ[ὸν ․․5․․]\n[․․․․․12․․․․․]ο...,Att.,stoich. 23,None,Ath.: Eleusinion,ca. 460 BC,ca. 460 BC,"IG I² 6,C","I(3).6,C ll. 5-50",None,None
1,/text/237767?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","18,H5",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 18,H5",None,None,35.0,\n1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n...,corona\nθ[εοί].\nΕ̣ὐθ̣[․․6․․․]θ̣ωνος Ἐλευσίνιο...,Att.,stoich. 25,None,Eleusis,ca. mid-4th c. BC,ca. mid-4th c. BC,IG II² 1188,None,None,None
2,/text/237768?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","24,H11",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 24,H11",None,None,26.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἐπὶ Λυσιάδου ἄρχοντος Π̣[υανοψ]ιῶνος ἕκτ[ει ἐπ...,Att.,non-stoich.,None,Ath.: Agora,None,148/7? BC,"Hsp.11.1942.293,58 [SEG 19.124]",None,None,None
3,/text/237769?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","30,H19",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 30,H19",None,None,5.0,1\n\n\n\n5,[Τ. Φλ]ά̣β. Πάντα[ινον]\n[Γαργ]ήττιον vac.\n[․...,Att.,None,None,Athens,ca. 100 AD?,ca. 100 AD?,"Hsp.35.1966.247,8 +","AD 25.1970.187,6 [SEG 23.114]",SEG 29.180,None
4,/text/237770?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","31,H20",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 31,H20",None,None,14.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[κ]α̣θ’ ὑπομνη[ματι]-\nσμὸν τῆς ἐξ Ἀ[ρείου]\nπ...,Att.,None,None,Eleusis,166/7 AD,166/7 AD,IG II² 3984,None,None,None


In [13]:
print(PHI["hdr1"].unique().tolist())

['Regions\nAttica (IG I-III)', 'Regions', 'Regions\nAsia Minor', 'Regions\nAsia Minor\nPhrygia', 'Regions\nAegean Islands, incl. Crete (IG XI-[XIII])\nChios ', 'Regions\nAegean Islands, incl. Crete (IG XI-[XIII])\nCrete ', 'Regions\nCyprus ([IG XV])\nunspecified subregion', 'Regions\nNorthern Greece (IG X)\nEpeiros, Illyria, and Dalmatia ', 'Regions\nAttica (IG I-III)\nAttica', 'Regions\nNorthern Greece (IG X)\nMacedonia ', 'Regions\nNorth Shore of the Black Sea\nunspecified subregion', 'Regions\nEgypt, Nubia and Cyrenaïca\nEgypt and Nubia', 'Regions\nCentral Greece (IG VII-IX)\nDelphi ', 'Regions\nAsia Minor\nCilicia and Isauria', 'Regions\nAsia Minor\nPamphylia', 'Regions\nAegean Islands, incl. Crete (IG XI-[XIII])\nCos and Calymna (IG XII,4)', 'Regions\nAsia Minor\nGalatia', 'Regions\nAegean Islands, incl. Crete (IG XI-[XIII])\nRhodes and S. Dodecanese (IG XII,1) ', 'Regions\nAsia Minor\nLydia', 'Regions\nSicily, Italy, and the West (IG XIV)\nSicily, Sardinia, and neighboring Island

In [15]:
len(PHI[PHI["hdr1"]=="Regions"])

49855

In [20]:
PHI[PHI["hdr1"]=="Regions"]["Book"].unique()

array(['Clara Rhodos', 'Chiron', 'CRAI', 'CIL III', 'CIG', 'CEG',
       'Byzantion', 'Berytus', 'BPhW', 'BP', 'BE', 'BCH', 'BCH Suppl.',
       'Audollent, Defix. Tab.', 'Athēna', 'ArchPap', 'ArchEph', 'AntCl',
       'AnnÉp', 'Anc. Macedonia', 'ASAA', 'AJPh', 'AJA', 'AEMÖ', 'AD',
       'ABSA', 'AAA', 'AA', 'I.Kition', 'Hesperia', 'Hellenika',
       'Grabdenkmäler', 'GVI', 'GIBM IV,2', 'GIBM II', 'Ergon',
       'Eretria XI', 'ILGR', 'IK Estremo oriente', 'IGR III', 'IGBulg V',
       'IGBulg IV', 'IG IX,1²', 'Mendel, Catalogue', 'MUSJ', 'MDAI(I)',
       'MDAI(A)', 'Mélanges Daux', 'LBW', 'LAAA', 'Klio', 'JRS', 'JHS',
       'JÖAI', 'Robert, Hellenica 8', 'Robert, Hellenica 11/12',
       'Robert, Coll. Froehner', 'RhM', 'RPh', 'RFIC', 'REG', 'REA', 'RA',
       'Pritchett, Studies', 'Polemon', 'Pfuhl-Möbius', 'PAAH', 'PAA',
       'OGIS', 'MonPiot', 'ZPE', 'Wilhelm, Neue Beiträge', 'TAPA',
       'Syria', 'Syll.³', 'StudClas', 'Spomenik', 'SbBerlin', 'SGLIBulg',
       'SEG'], dty

In [14]:
PHI[PHI["hdr1"]=="Regions"].head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4
47,/text/190454?location=1&patt=&bookid=242&booki...,Clara Rhodos,"2 (1932) 169,1",Regions,"Clara Rhodos 2 (1932) 169,1",None,None,1.0,1,"{²See RFIC 60.452,II.}²",Rhodos Isl.,None,None,Rhodos,None,"RFIC 60 (1932) 452, II",None,None,None,None
48,/text/190455?location=1&patt=&bookid=242&booki...,Clara Rhodos,"2 (1932) 170,2",Regions,"Clara Rhodos 2 (1932) 170,2",None,None,1.0,1,{²See IG XII Suppl. 150 (Astypalaia).}²,Aegean Islands,None,None,Astypalaia,None,IG XII Suppl. 150,None,None,None,None
49,/text/190457?location=1&patt=&bookid=242&booki...,Clara Rhodos,"2 (1932) 175,4",Regions,"Clara Rhodos 2 (1932) 175,4",None,None,17.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n,[ἔδο]ξε {²corona}² Ἁγητορίδαις·\n[Θε]όμ...,Rhodos Isl.,None,None,prov. unkn.,2nd c. BC,2nd c. BC,"cf. ASAA 22 (1942) 176, n. 2",None,None,None
50,/text/190458?location=1&patt=&bookid=242&booki...,Clara Rhodos,"2 (1932) 176,5",Regions,"Clara Rhodos 2 (1932) 176,5",None,None,39.0,col. I.1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[— — —] Πυθέα\n[— — —]ριστας\n[— — —] Νικοπόλι...,Rhodos Isl.,None,None,Rhodos,late 3rd/2nd c. BC,late 3rd/2nd c. BC,cf. GGA 195 (1933) 21-22,None,None,None
51,/text/190459?location=1&patt=&bookid=242&booki...,Clara Rhodos,"2 (1932) 177,6",Regions,"Clara Rhodos 2 (1932) 177,6",None,None,76.0,1\n\n\n\ncol. I.5\n\n\n\n\n10\n\n\n\n\n15\n\n\...,[— — — — — — — —]ελομεν[— — — — — — — — — — — ...,Rhodos Isl.,None,None,Rhodos,1st c. BC,1st c. BC,None,None,None,None


In [0]:
PHI["location"] = PHI.apply(lambda row: row["URL"].rpartition("location=")[2], axis=1)

In [0]:
PHI["location_first"] = PHI.apply(lambda row: row["URL"].partition("location=")[2].partition("&")[0], axis=1)

In [40]:
PHI.head(50)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4,location,location_first
0,/text/237766?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials",10,Regions\nAttica (IG I-III),"Clinton, Sacred Officials 10",None,None,46.0,5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20\n\n\n\n\...,[․․․․․12․․․․․] ὀβολ[ὸν ․․5․․]\n[․․․․․12․․․․․]ο...,Att.,stoich. 23,None,Ath.: Eleusinion,ca. 460 BC,ca. 460 BC,"IG I² 6,C","I(3).6,C ll. 5-50",None,None,7,7
1,/text/237767?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","18,H5",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 18,H5",None,None,35.0,\n1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n...,corona\nθ[εοί].\nΕ̣ὐθ̣[․․6․․․]θ̣ωνος Ἐλευσίνιο...,Att.,stoich. 25,None,Eleusis,ca. mid-4th c. BC,ca. mid-4th c. BC,IG II² 1188,None,None,None,7,7
2,/text/237768?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","24,H11",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 24,H11",None,None,26.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἐπὶ Λυσιάδου ἄρχοντος Π̣[υανοψ]ιῶνος ἕκτ[ει ἐπ...,Att.,non-stoich.,None,Ath.: Agora,None,148/7? BC,"Hsp.11.1942.293,58 [SEG 19.124]",None,None,None,7,7
3,/text/237769?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","30,H19",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 30,H19",None,None,5.0,1\n\n\n\n5,[Τ. Φλ]ά̣β. Πάντα[ινον]\n[Γαργ]ήττιον vac.\n[․...,Att.,None,None,Athens,ca. 100 AD?,ca. 100 AD?,"Hsp.35.1966.247,8 +","AD 25.1970.187,6 [SEG 23.114]",SEG 29.180,None,7,7
4,/text/237770?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","31,H20",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 31,H20",None,None,14.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n,[κ]α̣θ’ ὑπομνη[ματι]-\nσμὸν τῆς ἐξ Ἀ[ρείου]\nπ...,Att.,None,None,Eleusis,166/7 AD,166/7 AD,IG II² 3984,None,None,None,7,7
5,/text/237771?location=7&patt=&bookid=409&offse...,"Clinton, Sacred Officials","39,H25",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 39,H25",None,None,4.0,1\n\n\n,ἀρρήτων θησ[αυρὸν ⏑–⏑⏑– ἐ]ς̣ Ἀθήνας\nμυστικὸν ...,Att.,None,None,Eleusis,ca. 170 AD,ca. 170 AD,ASAA 37-38.1959/60.421-427,SEG 22.156; SEG 30.339,None,None,7,7
6,/text/237772?location=7&patt=&bookid=409&offse...,"Clinton, Sacred Officials","39,H26",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 39,H26",None,None,6.0,4\n5\n\n\n\n,... [ὁ κοσ]μ̣(?)ητὴς\nΚλαύδιο[ς Πο]λύζηλος\nἈχ...,Att.,None,None,Athens,194/5 AD,194/5 AD,"IG II² 2109, ll. 4, 9",None,None,None,7,7
7,/text/237773?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","40,H27",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 40,H27",None,None,3.0,1\n\n,Νούμμιος Ἱερο[φ]ά̣ν̣[της]\nΝούμ(μιος) Ἱεροκ̣ῆ̣...,Att.,None,None,Athens,ca. 197 AD,ca. 197 AD,IG II² 1790 ll. 26-27,Ag. 15.427 II ll. 30-33,None,None,7,7
8,/text/237774?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","41,H29",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 41,H29",None,None,4.0,1\n\n\n,[κατὰ τὰ] δό̣ξαντα Ἀρ[εοπαγείταις]\n[․c.5․] Ἀ̣...,Att.,None,None,Ath.: Asklepieion,beg. 3rd c. AD,beg. 3rd c. AD,IG II² 3812,None,None,None,7,7
9,/text/237775?location=7&patt=&bookid=409&booki...,"Clinton, Sacred Officials","50,D14",Regions\nAttica (IG I-III),"Clinton, Sacred Officials 50,D14",None,None,69.0,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἀγαθῆι τύχηι τῆς βουλῆς καὶ̣ τ̣[οῦ δήμο]υ τοῦ ...,Att.,None,None,Eleusis,20/19 BC,20/19 BC,Kourouniotes (1932) 223-236,SEG 30.93,None,None,7,7


In [41]:
[reg for reg in PHI["location_first"].unique().tolist() if len(reg)<3]

['7', '1', '14', '21', '3', '6', '54', '8', '12', '37', '11', '31', '16']

In [38]:
[reg for reg in PHI["location"].unique().tolist() if len(reg)<3]

['7', '1', '14', '21', '3', '6', '54', '8', '12', '37', '11', '31', '16']

In [44]:
PHI.groupby("location").size()

location
1       49855
1002      605
1028      557
1040     6711
1090     1013
        ...  
789      2384
8        4027
850         8
915      3372
941      4539
Length: 72, dtype: int64

In [42]:
PHI.groupby("location_first").size()

location_first
1       49855
1002      605
1028      557
1040     6711
1090     1013
        ...  
789      2384
8        4027
850         8
915      3372
941      4539
Length: 72, dtype: int64

In [29]:
PHI[PHI["URL"].str.contains("190454")]

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4
47,/text/190454?location=1&patt=&bookid=242&booki...,Clara Rhodos,"2 (1932) 169,1",Regions,"Clara Rhodos 2 (1932) 169,1",None,None,1.0,1,"{²See RFIC 60.452,II.}²",Rhodos Isl.,None,None,Rhodos,None,"RFIC 60 (1932) 452, II",None,None,None,None


In [17]:
PHI[PHI["hdr1"]=="Regions"]["subregion"].unique()

array(['Rhodos', 'Astypalaia', 'prov. unkn.', ..., 'Pınarlı Köy, area of',
       'Thyaira (Tire)', 'Prov. unkn. [Kos, Kastro]'], dtype=object)

In [28]:
sorted([el for el in PHI["region"].unique().tolist() if el != None])

['2. Mil-Ber. 911',
 '2. Mil-Ber. 911-12',
 '3. Mil-Ber. 83-84',
 '3. Mil-Ber. 85',
 '3. Mil-Ber. 87',
 '3. Mil-Ber. 89',
 '4. Mil-Ber. 533-34',
 '4. Mil-Ber. 534-35',
 '4. Mil-Ber. 535',
 '4. Mil-Ber. 547',
 '6. Mil-Ber. 20-22',
 '6. Mil-Ber. 22',
 '6. Mil-Ber. 22-23',
 '6. Mil-Ber. 23-24',
 'AA 100, 1985, 445-46, no. 1',
 'AA 100, 1985, 446-47, no. 2',
 'AA 100, 1985, 447, no. 3',
 'AA 100, 1985, 447-48, no. 4',
 'AA 100, 1985, 448-49, no. 5',
 'AA 79, 1964, 537-42, no. 31',
 'AA 80, 1965, 439',
 'AA 80, 1965, 440',
 'AA 93, 1970, 255-56',
 'AA 93, 1978, 254-55',
 'AA 95, 1980, 349-50',
 'AAA 13, 1980, 310',
 'AAA 13, 1980, 310, no. 1',
 'AAA 13, 1980, 310-11, no. 2',
 'AAA 13, 1980, 311, no. 3',
 'AAA 13, 1980, 311, no. 4',
 'AAWW 116, 1979, 130',
 'AAWW 120, 1983, Taf. X',
 'AAWW 1893, 101, no. 5',
 'AAWW 1893, 102, no. 11',
 'AArch 35, 1964, 202-3',
 'ABSA 1926-27, 117, no. 1',
 'ABSA 1926-27, 117-18, no. 2',
 'ABSA 1926-27, 118, no. 3',
 'ABSA 1926-27, 127, no. 19',
 'ABSA 1926-2